### Dataset 출처

> RAG & LLM 오픈소스

- RAG 기반 비구조화된 데이터를 기반으로 질문에 답변하는 오픈 소스 : https://huggingface.co/learn/cookbook/rag_with_unstructured_data

- 다양한 유형의 소스 (PDF, YouTube 동영상) 로부터 데이터를 가공해 RAG 파이프 라인을 구현하는 예제의 컬럼 :  https://huggingface.co/blog/ngxson/make-your-own-rag

---

> DipLearning

- ResNet을 이용한 개 고양이 분류기 : https://github.com/woodong11/Cat_vs_Dog_classifier/blob/main/train.ipynb

- GAN을 이용한 MNIST 숫자 생성 모델 : https://github.com/happy-jihye/Awesome-GAN-Papers/blob/main/gan/gan.ipynb

---

> MachineLearning

- ETF 예측 모델 (다중선형회귀, XGBoost, ARIMA) :  https://github.com/terri1102/Stock_price_prediction/blob/main/stock_price_prediction.ipynb


---

> Data anaysis

- 서울시 공공 자전거 분석 : https://github.com/coding-Benny/data-visualization/blob/master/public_bicycle.ipynb

- 무더위 쉼터 데이터 : https://github.com/coding-Benny/data-visualization/blob/master/heatwave_shelter_service.ipynb


### 1. URL, user agent 전역 선언

In [2]:
import requests
from bs4 import BeautifulSoup
import json
from langchain_community.document_loaders import WebBaseLoader


url_rag_1 = "https://huggingface.co/learn/cookbook/rag_with_unstructured_data"
url_rag_2 = "https://huggingface.co/blog/ngxson/make-your-own-rag"
url_dl_1 = "https://github.com/woodong11/Cat_vs_Dog_classifier/blob/main/train.ipynb"
url_dl_2 = "https://github.com/happy-jihye/Awesome-GAN-Papers/blob/main/gan/gan.ipynb"
url_ml_1 = "https://github.com/terri1102/Stock_price_prediction/blob/main/stock_price_prediction.ipynb"
url_anaysis_1 = "https://github.com/coding-Benny/data-visualization/blob/master/public_bicycle.ipynb"
url_anaysis_2 = "https://github.com/coding-Benny/data-visualization/blob/master/heatwave_shelter_service.ipynb"

headers = {
    "User-Agent": "yMozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"  
}

USER_AGENT environment variable not set, consider setting it to identify your requests.


#### 2-1. RAG & LLM 데이터 가공 1: code block과 본문을 분리해서 저장하는 방법

In [69]:
# code block만을 파싱하는 함수

def get_code_block_from_huggingface(url:str, header:dict) -> list:
    # 웹 페이지 요청
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # `pre` 태그 모두 찾기
    pre_tags = soup.find_all('pre')

    # pre 태그의 텍스트를 리스트로 추출
    pre_texts = [pre.get_text() for pre in pre_tags]

    return pre_texts

In [70]:
# 본문만을 파싱하는 함수

def get_content_from_huggingface(url:str, header:dict) -> list:
    # 웹 페이지 요청
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    # `pre` 태그 모두 찾기
    pre_tags = soup.find_all('p')

    # pre 태그의 텍스트를 리스트로 추출
    pre_texts = [pre.get_text() for pre in pre_tags]

    return pre_texts

#### RAG & LLM 전용 Json 타입으로 저장하는 메서드

In [9]:
import os
import json

# 기본 저장 경로
root_path = "open_source"

def get_partition_and_save_to_json(url: str, header: str, path: str, file_name: str):
    try:
        # Huggingface URL에서 코드블럭과 콘텐츠를 추출
        code_block = get_code_block_from_huggingface(url, header)
        content = get_content_from_huggingface(url, header)

        # 저장 디렉토리 생성 (없으면 생성)
        save_path = os.path.join(root_path, path)
        os.makedirs(save_path, exist_ok=True)

        # 코드블럭 JSON 저장
        code_file_path = os.path.join(save_path, f"{file_name}_codeblock.json")
        with open(code_file_path, 'w', encoding='utf-8') as code_file:
            json.dump(code_block, code_file, ensure_ascii=False, indent=4)

        # 콘텐츠 JSON 저장
        content_file_path = os.path.join(save_path, f"{file_name}_content.json")
        with open(content_file_path, 'w', encoding='utf-8') as content_file:
            json.dump(content, content_file, ensure_ascii=False, indent=4)

        print(f"JSON 파일 저장 완료:\n- 코드블럭: {code_file_path}\n- 콘텐츠: {content_file_path}")

    except Exception as e:
        print(f"JSON 저장 중 오류 발생: {e}")


In [72]:
get_partition_and_save_to_json(url_rag_1, headers, "partition/llm_rag", "unconstructed_data")
get_partition_and_save_to_json(url_rag_2, headers, "partition/llm_rag", "various_source")

JSON 파일 저장 완료:
- 코드블럭: open_source/partition/llm_rag/unconstructed_data_codeblock.json
- 콘텐츠: open_source/partition/llm_rag/unconstructed_data_content.json
JSON 파일 저장 완료:
- 코드블럭: open_source/partition/llm_rag/various_source_codeblock.json
- 콘텐츠: open_source/partition/llm_rag/various_source_content.json


### 2-2. WebBaseLoader 이용 (codeblock + text 모두 포함됨) - RAW 한 데이터

In [11]:
import re
import os

def raw_preProcess(text):
    # 2. 소문자 변환
    text = text.lower()

    # \n과 \t 공간 삭제
    text = re.sub(r'[\n\t]', '', text)

    # 5. 전처리된 텍스트 반환
    return ''.join(text)


In [15]:
def get_webBaseLoader_and_save_to_text(url:str, path: str, file_name:str):

    try:
        loader = WebBaseLoader(
            web_paths= [url]
        )

        docs = loader.load()
        result = ""
        for doc in docs:
            result += raw_preProcess(doc.page_content)

        # 저장 디렉토리 생성 (없으면 생성)
        save_path = os.path.join(root_path, path)
        os.makedirs(save_path, exist_ok=True)

        # 코드블럭 JSON 저장
        raw_file_path = os.path.join(save_path, f"{file_name}_raw.txt")

        with open(raw_file_path, 'w', encoding='utf-8') as content_file:
            content_file.write(result)
            
        print(f"JSON 파일 저장 완료:\n- raw: {raw_file_path}")

    except Exception as e:
        print(f"JSON 저장 중 오류 발생: {e}")


In [16]:
get_webBaseLoader_and_save_to_text(url_rag_1, "raw/llm_rag", "unconstructed_data")
get_webBaseLoader_and_save_to_text(url_rag_1, "raw/llm_rag", "various_source")

JSON 파일 저장 완료:
- raw: open_source/raw/llm_rag/unconstructed_data_raw.txt
JSON 파일 저장 완료:
- raw: open_source/raw/llm_rag/various_source_raw.txt


#### 2-3. github의 ipynb파일 가져오기

In [22]:
! pip install nbformat

In [14]:
def convert_to_raw_url(github_url: str) -> str:
    if "github.com" in github_url:
        # 'blob'을 '/'로 바꾸고 'github.com'을 'raw.githubusercontent.com'으로 변환
        raw_url = github_url.replace("github.com", "raw.githubusercontent.com").replace("/blob", "")
        return raw_url
    else:
        raise ValueError("Invalid GitHub URL")

In [17]:
import requests
import nbformat

def get_code_ipynb_from_github(url: str, headers: dict = None) -> list:
    response = requests.get(convert_to_raw_url(url), headers=headers)
    result = []
    
    if response.status_code == 200:
        try:
            # 텍스트로 데이터를 가져온 뒤 JSON 파싱
            notebook_content = response.text
            notebook = nbformat.reads(notebook_content, as_version=4)

            # Notebook 셀 확인
            for cell in notebook.get('cells', []):
                if cell.get('cell_type') == 'code':  # 코드 셀인지 확인
                    result.append(cell.get('source', ""))
                
            return result
        except Exception as e:
            print(f"Notebook 처리 중 오류 발생: {e}")
    else:
        print(f"Failed to fetch notebook: HTTP {response.status_code}")



In [63]:
def get_markdown_ipynb_from_github(url:str, header:str) -> list:

    response = requests.get(convert_to_raw_url(url))
    result = []
    if response.status_code == 200:
        notebook_content = response.text
        # JSON 형식의 notebook 파일 로드
        notebook = nbformat.reads(notebook_content, as_version=4)
        
        # Notebook 셀 확인
        for cell in notebook.get('cells', []):
            # 이미지나 다른 데이터가 포함된 출력은 제외
            if cell.get('cell_type') == 'markdown':
                if 'image' not in cell.get('source', '').lower():
                    result.append(cell.get('source', ""))
                
            elif 'outputs' in cell:
                # 출력에 이미지가 포함되지 않도록 필터링
                outputs = cell['outputs']
                filtered_outputs = [
                    output for output in outputs if output.output_type != 'display_data' or 'image' not in output.data
                ]
                if filtered_outputs:
                    result.extend([output.get('text/plain', '') for output in filtered_outputs])
                        
        return result
    else:
        print(f"Failed to fetch notebook: {response.status_code}")

In [64]:
import os
# 기본 저장 경로
root_path = "open_source"

def save_to_json_github(url: str, header: str, path: str, file_name: str):
    try:
        # Huggingface URL에서 코드블럭과 콘텐츠를 추출
        code_block = get_code_ipynb_from_github(url, header)
        content = get_markdown_ipynb_from_github(url, header)

        # 저장 디렉토리 생성 (없으면 생성)
        save_path = os.path.join(root_path, path)
        os.makedirs(save_path, exist_ok=True)

        # 코드블럭 JSON 저장
        code_file_path = os.path.join(save_path, f"{file_name}_codeblock.json")
        with open(code_file_path, 'w', encoding='utf-8') as code_file:
            json.dump(code_block, code_file, ensure_ascii=False, indent=4)

        # 콘텐츠 JSON 저장
        content_file_path = os.path.join(save_path, f"{file_name}_content.json")
        with open(content_file_path, 'w', encoding='utf-8') as content_file:
            json.dump(content, content_file, ensure_ascii=False, indent=4)

        print(f"JSON 파일 저장 완료:\n- 코드블럭: {code_file_path}\n- 콘텐츠: {content_file_path}")

    except Exception as e:
        print(f"JSON 저장 중 오류 발생: {e}")

In [46]:
save_to_json_github(url_dl_1, headers, "partition/dl", "cat_and_dog")

In [67]:
save_to_json_github(url_anaysis_1, headers, "partition/analysis", "bicycle")
save_to_json_github(url_anaysis_2, headers, "partition/analysis", "heatware")

JSON 파일 저장 완료:
- 코드블럭: open_source/partition/analysis/bicycle_codeblock.json
- 콘텐츠: open_source/partition/analysis/bicycle_content.json
JSON 파일 저장 완료:
- 코드블럭: open_source/partition/analysis/heatware_codeblock.json
- 콘텐츠: open_source/partition/analysis/heatware_content.json


In [19]:
def get_raw_ipynb_from_github(url: str, headers: dict = None) -> str:
    response = requests.get(convert_to_raw_url(url), headers=headers)
    result = ""  # 빈 문자열로 수정
    
    if response.status_code == 200:
        try:
            # 텍스트로 데이터를 가져온 뒤 JSON 파싱
            notebook_content = response.text
            notebook = nbformat.reads(notebook_content, as_version=4)

            # Notebook 셀 확인
            for cell in notebook.get('cells', []):
                if cell.get('cell_type') == 'code':  # 코드 셀만 선택
                    result += raw_preProcess(cell.get('source', ""))
                
                # 이미지나 다른 데이터가 포함된 출력은 제외
                elif cell.get('cell_type') == 'markdown':
                    if 'image' not in cell.get('source', '').lower():
                        result += raw_preProcess(cell.get('source', ""))
                
                elif 'outputs' in cell:
                    # 출력에 이미지가 포함되지 않도록 필터링
                    outputs = cell['outputs']
                    filtered_outputs = [
                        output for output in outputs if output.output_type != 'display_data' or 'image' not in output.data
                    ]
                    if filtered_outputs:
                        # `extend()`를 `+=`로 변경하여 문자열을 이어 붙이기
                        result += "".join([output.get('text/plain', '') for output in filtered_outputs])

            return result
        except Exception as e:
            print(f"Notebook 처리 중 오류 발생: {e}")
    else:
        print(f"Failed to fetch notebook: HTTP {response.status_code}")


In [21]:
import os
# 기본 저장 경로
root_path = "open_source"

def save_raw_to_text_github(url: str, header: str, path: str, file_name: str):
    try:
        # Huggingface URL에서 코드블럭과 콘텐츠를 추출
        result = get_raw_ipynb_from_github(url, header)

        # 저장 디렉토리 생성 (없으면 생성)
        save_path = os.path.join(root_path, path)
        os.makedirs(save_path, exist_ok=True)

        # 콘텐츠 JSON 저장
        # 코드블럭 JSON 저장
        raw_file_path = os.path.join(save_path, f"{file_name}_raw.txt")

        with open(raw_file_path, 'w', encoding='utf-8') as content_file:
            content_file.write(result)

        print(f"JSON 파일 저장 완료:\n- raw: {raw_file_path}")

    except Exception as e:
        print(f"JSON 저장 중 오류 발생: {e}")

In [24]:
save_raw_to_text_github(url_ml_1, headers, "raw/dl", "stock")

JSON 파일 저장 완료:
- raw: open_source/raw/dl/stock_raw.txt
